# RITAL - Bag of Words Project

Binôme :
- **Ben KABONGO**, 21116436

- **Sofia BORCHANI**, 21212080

## Partie I : Données reconnaissance du locuteur (Chirac/Mitterrand) : variantes de BoW et machine learning

**Problématiques**

- **Variantes de BoW**
    - TF-IDF
    - Réduire la taille du vocabulaire (min_df, max_df, max_features)
    - BoW binaire
    - Bi-grams, tri-grams
    - **Quelles performances attendrent? Quels sont les avantages et les inconvénients des ces variantes?**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/RITAL/rital-BoW-Project/BoW-Project/locuteurs/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
/content/gdrive/My Drive/RITAL/rital-BoW-Project/BoW-Project/locuteurs


In [2]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import sklearn
import warnings

from imblearn.under_sampling import RandomUnderSampler

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn import (linear_model, 
                     metrics,
                     preprocessing, 
                     model_selection, 
                     pipeline,)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from xgboost import XGBClassifier

import utils

In [3]:
plt.style.use('seaborn-whitegrid')
warnings.simplefilter("ignore")

<ipython-input-3-61454a7c5a18>:1: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


## Chargement des données

In [4]:
fname = "./datasets/AFDpresidentutf8/corpus.tache1.learn.utf8"
all_locuteur_df = utils.load_pres(fname)
all_locuteur_df

,text,label
0,"Quand je dis chers amis, il ne s'agit pas là ...",1
1,D'abord merci de cet exceptionnel accueil que...,1
2,C'est toujours très émouvant de venir en Afri...,1
3,Aucun citoyen français ne peut être indiffére...,1
4,"Le Congo, que naguère le <nom> qualifia de ""r...",1
...,...,...
57408,Je suis heureux de le mener avec vous.\n,1
57409,"Vous le savez, comme vous, j'ai la passion de...",1
57410,Je crois en son avenir.\n,1
57411,"Je crois en la politique, c'est-à-dire en not...",1


In [5]:
fname = "./datasets/AFDpresidentutf8/corpus.tache1.test.utf8"
all_locuteur_test_df = utils.load_pres_test(fname)
all_locuteur_test_df

,text
0,"En répondant à votre invitation, en effectuant..."
1,"Et ce moment exceptionnel, j'ai essayé de le c..."
2,Et le musée.
3,Je parlais en même temps que le Maire d'Angers...
4,<nom> devait plus tard n'y être pas pour rien ...
...,...
27157,Mais nous devons tout faire pour les aider à d...
27158,"Pour cela, nous devons d'abord utiliser pleine..."
27159,"Mais, face aux enjeux énergétiques des pays en..."
27160,Le prochain sommet du G8 nous en donne l'occas...


## Méthodes de machine learning et variantes de BoW

In [6]:
y = all_locuteur_df.text

In [7]:
np.random.seed(0)

In [13]:
def lvectorizer(vectorizer):
    X_text_train, X_text_test, y_train, y_test = model_selection.train_test_split(all_locuteur_df['text'], 
        all_locuteur_df['label'], test_size=0.2, random_state=42)

    X_train = vectorizer.fit_transform(X_text_train)
    X_test = vectorizer.transform(X_text_test)
    
    rus = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

    clf = linear_model.LogisticRegression()
    clf.fit(X_resampled, y_resampled)

    y_pred = clf.predict(X_test)

    y_prob = clf.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    auc = metrics.auc(fpr, tpr)
    
    f1 = metrics.f1_score(y_test, y_pred)
    acc = metrics.accuracy_score(y_test, y_pred)
    bacc = metrics.balanced_accuracy_score(y_test, y_pred)
    precision, recall, f1_score, support = metrics.precision_recall_fscore_support(y_test, y_pred)

    report = metrics.classification_report(y_test, y_pred)
    print(report)

    print()
    print("F1-score -1\t:", f1_score[0])
    print('Accuracy \t:', acc)
    print('Bal. acc \t:', bacc)
    print('F1 score \t:', f1)
    print('AUC \t\t:', auc)

    

### CountVectorizer

In [9]:
def clvectorizer(**count_vectorizer_args):
    vectorizer = CountVectorizer(**count_vectorizer_args)
    return lvectorizer(vectorizer)

### TfidfVectorizer

In [14]:
def tlvectorizer(**tfidf_vectorizer_args):
    vectorizer = TfidfVectorizer(**tfidf_vectorizer_args)
    return lvectorizer(vectorizer)

### Variantes et évaluations

In [15]:
clvectorizer()

              precision    recall  f1-score   support

          -1       0.36      0.76      0.49      1481
           1       0.96      0.80      0.87     10002

    accuracy                           0.79     11483
   macro avg       0.66      0.78      0.68     11483
weighted avg       0.88      0.79      0.82     11483


F1-score -1	: 0.48625243559211956
Accuracy 	: 0.7933466864059915
Bal. acc 	: 0.7784058650795163
F1 score 	: 0.8706600534147272
AUC 		: 0.8604100246797365


In [12]:
tlvectorizer()

              precision    recall  f1-score   support

          -1       0.36      0.75      0.49      1481
           1       0.96      0.80      0.87     10002

    accuracy                           0.80     11483
   macro avg       0.66      0.78      0.68     11483
weighted avg       0.88      0.80      0.82     11483


F1-score -1	:  0.48861646234676015
Accuracy 	: 0.7965688408952364
Bal. acc 	: 0.7782421571053784
F1 score 	: 0.8730296771388194
AUC 		: 0.8580330524036988


In [17]:
#Test avec la suppression de la ponctuation :
clvectorizer(preprocessor=utils.delete_punctuation)

              precision    recall  f1-score   support

          -1       0.37      0.76      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.80     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.80      0.83     11483


F1-score -1	: 0.4997782705099778
Accuracy 	: 0.8035356614125229
Bal. acc 	: 0.7854051741981111
F1 score 	: 0.8777633289986996
AUC 		: 0.8708709979813625


In [18]:
#Test avec la suppression de la ponctuation :
tlvectorizer(preprocessor=utils.delete_punctuation)

              precision    recall  f1-score   support

          -1       0.38      0.77      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.81     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.81      0.83     11483


F1-score -1	: 0.5037760995113282
Accuracy 	: 0.8054515370547767
Bal. acc 	: 0.7885182922902252
F1 score 	: 0.8790077989601385
AUC 		: 0.8670862046361828


In [19]:
#Test avec la suppression des chiffres :
clvectorizer(preprocessor=utils.delete_digit)

              precision    recall  f1-score   support

          -1       0.37      0.76      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.80     11483
   macro avg       0.66      0.79      0.69     11483
weighted avg       0.88      0.80      0.83     11483


F1-score -1	: 0.49933540097474516
Accuracy 	: 0.8031873203866585
Bal. acc 	: 0.7852052141901127
F1 score 	: 0.8775200520268807
AUC 		: 0.8706035970388637


In [20]:
#Test avec la suppression des chiffres :
tlvectorizer(preprocessor=utils.delete_digit)

              precision    recall  f1-score   support

          -1       0.37      0.77      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.80     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.80      0.83     11483


F1-score -1	: 0.5022163120567377
Accuracy 	: 0.8044065139771837
Bal. acc 	: 0.7876307925450696
F1 score 	: 0.8782919692207651
AUC 		: 0.8669601663732075


In [21]:
clvectorizer(preprocessor=utils.delete_balise)

              precision    recall  f1-score   support

          -1       0.37      0.76      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.80     11483
   macro avg       0.66      0.79      0.69     11483
weighted avg       0.88      0.80      0.83     11483


F1-score -1	: 0.4988933156263834
Accuracy 	: 0.8028389793607942
Bal. acc 	: 0.7850052541821143
F1 score 	: 0.8772766695576756
AUC 		: 0.8710263349085754


In [22]:
tlvectorizer(preprocessor=utils.delete_balise)

              precision    recall  f1-score   support

          -1       0.38      0.77      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.81     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.81      0.83     11483


F1-score -1	: 0.5043362241494329
Accuracy 	: 0.8058869633371071
Bal. acc 	: 0.7887682423002232
F1 score 	: 0.8793112783583302
AUC 		: 0.8674498051098761


In [23]:
tlvectorizer(preprocessor=utils.first_line)

              precision    recall  f1-score   support

          -1       0.37      0.77      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.80     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.80      0.83     11483


F1-score -1	: 0.500220361392684
Accuracy 	: 0.80249063833493
Bal. acc 	: 0.7871062519433993
F1 score 	: 0.8769264163229867
AUC 		: 0.8640774883510806


In [24]:
tlvectorizer(preprocessor=utils.last_line)

              precision    recall  f1-score   support

          -1       0.36      0.75      0.49      1481
           1       0.96      0.80      0.87     10002

    accuracy                           0.80     11483
   macro avg       0.66      0.78      0.68     11483
weighted avg       0.88      0.80      0.82     11483


F1-score -1	: 0.48814749780509215
Accuracy 	: 0.7969171819211007
Bal. acc 	: 0.7772916382287351
F1 score 	: 0.873329712112982
AUC 		: 0.8601614248385975


In [26]:
nltk.download('punkt')
tlvectorizer(preprocessor=utils.stem)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


              precision    recall  f1-score   support

          -1       0.36      0.76      0.49      1481
           1       0.96      0.80      0.87     10002

    accuracy                           0.80     11483
   macro avg       0.66      0.78      0.68     11483
weighted avg       0.88      0.80      0.82     11483


F1-score -1	: 0.4882608695652173
Accuracy 	: 0.795001306278847
Bal. acc 	: 0.7793556751175086
F1 score 	: 0.8718283785255363
AUC 		: 0.8571646237936748


In [27]:
tlvectorizer(preprocessor=utils.replace_maj_word)

              precision    recall  f1-score   support

          -1       0.37      0.76      0.50      1481
           1       0.96      0.81      0.88     10002

    accuracy                           0.80     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.80      0.83     11483


F1-score -1	: 0.5016648168701443
Accuracy 	: 0.8044935992336497
Bal. acc 	: 0.7868179233835879
F1 score 	: 0.8783922864416879
AUC 		: 0.8662103500974349


In [28]:
tlvectorizer(binary=True)

              precision    recall  f1-score   support

          -1       0.36      0.75      0.49      1481
           1       0.96      0.80      0.87     10002

    accuracy                           0.80     11483
   macro avg       0.66      0.78      0.68     11483
weighted avg       0.88      0.80      0.82     11483


F1-score -1	: 0.4884590019784568
Accuracy 	: 0.7973526082034311
Bal. acc 	: 0.7772539685175727
F1 score 	: 0.8736493457131997
AUC 		: 0.8604033413438852


In [29]:
tlvectorizer(ngram_range=(2, 2))

              precision    recall  f1-score   support

          -1       0.34      0.75      0.46      1481
           1       0.95      0.78      0.86     10002

    accuracy                           0.78     11483
   macro avg       0.64      0.76      0.66     11483
weighted avg       0.87      0.78      0.81     11483


F1-score -1	: 0.46295136714673346
Accuracy 	: 0.7759296351127755
Bal. acc 	: 0.7643811885833502
F1 score 	: 0.8584319119669876
AUC 		: 0.8415385120140049


In [30]:
tlvectorizer(ngram_range=(1, 2))

              precision    recall  f1-score   support

          -1       0.35      0.77      0.48      1481
           1       0.96      0.79      0.86     10002

    accuracy                           0.79     11483
   macro avg       0.65      0.78      0.67     11483
weighted avg       0.88      0.79      0.82     11483


F1-score -1	: 0.48107653490328006
Accuracy 	: 0.7850735870417138
Bal. acc 	: 0.7796968290339231
F1 score 	: 0.8644700713893466
AUC 		: 0.861053582666316


In [31]:
tlvectorizer(ngram_range=(2, 3))

              precision    recall  f1-score   support

          -1       0.33      0.76      0.46      1481
           1       0.96      0.77      0.85     10002

    accuracy                           0.77     11483
   macro avg       0.64      0.77      0.66     11483
weighted avg       0.88      0.77      0.80     11483


F1-score -1	: 0.4611282051282051
Accuracy 	: 0.771227031263607
Bal. acc 	: 0.7659960242927781
F1 score 	: 0.8547896744237468
AUC 		: 0.8393020923161755


In [32]:
tlvectorizer(max_features=5000)

              precision    recall  f1-score   support

          -1       0.36      0.75      0.49      1481
           1       0.96      0.80      0.87     10002

    accuracy                           0.79     11483
   macro avg       0.66      0.78      0.68     11483
weighted avg       0.88      0.79      0.82     11483


F1-score -1	: 0.48593238822246454
Accuracy 	: 0.7947400505094487
Bal. acc 	: 0.7766171276210659
F1 score 	: 0.871769762254502
AUC 		: 0.8559349912596819


In [33]:
tlvectorizer(min_df=10, max_df=0.75, binary=True, ngram_range=(1, 2), preprocessor=utils.delete_punctuation)

              precision    recall  f1-score   support

          -1       0.38      0.75      0.51      1481
           1       0.96      0.82      0.88     10002

    accuracy                           0.81     11483
   macro avg       0.67      0.79      0.69     11483
weighted avg       0.88      0.81      0.83     11483


F1-score -1	: 0.5066908596053527
Accuracy 	: 0.8105895671862754
Bal. acc 	: 0.7865781671484744
F1 score 	: 0.8827935549927252
AUC 		: 0.8729149511083603


In [34]:
tlvectorizer(preprocessor=lambda x: utils.delete_punctuation(utils.first_line(utils.delete_balise(x))), binary=True, ngram_range=(3, 3), max_df=0.5)

              precision    recall  f1-score   support

          -1       0.31      0.67      0.42      1481
           1       0.94      0.78      0.85     10002

    accuracy                           0.76     11483
   macro avg       0.62      0.72      0.64     11483
weighted avg       0.86      0.76      0.80     11483


F1-score -1	: 0.42293753997015565
Accuracy 	: 0.7642602107463207
Bal. acc 	: 0.7240310209396339
F1 score 	: 0.85187414500684
AUC 		: 0.7955499717072116


In [35]:
tlvectorizer(preprocessor=lambda x: utils.stem(utils.delete_digit(utils.delete_punctuation(x))), binary=True, ngram_range=(1, 3))

              precision    recall  f1-score   support

          -1       0.33      0.79      0.47      1481
           1       0.96      0.77      0.85     10002

    accuracy                           0.77     11483
   macro avg       0.65      0.78      0.66     11483
weighted avg       0.88      0.77      0.80     11483


F1-score -1	: 0.4689627553063676
Accuracy 	: 0.769049899851955
Bal. acc 	: 0.7782644011373283
F1 score 	: 0.8524371244157578
AUC 		: 0.861168279510877
